<a href="https://colab.research.google.com/github/mhanauer/tempus_case_study_22/blob/main/tempus_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First I will clone the repository I created.

In [2]:
!git clone https://github.com/mhanauer/tempus_case_study_22.git

Cloning into 'tempus_case_study_22'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


Next I will install all the necesary python packages.

In [1]:
!pip install xgboost
!pip install lightgbm
!pip install scikit-optimize
!pip install skimpy
import pandas as pd
import sklearn
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import numpy as np
from joblib import dump, load
from sklearn.model_selection import RepeatedKFold
import json
from pandas.core.algorithms import duplicated
from skimpy import clean_columns
%cd tempus_case_study_22/

/content/tempus_case_study_22


Next load in the data and combine the biomarkers.

In [2]:
biomarkers_1 = pd.read_csv("biomarkers_1.csv")
biomarkers_2 = pd.read_csv("biomarkers_2.csv")
biomarkers_3 = pd.read_csv("biomarkers_3.csv")
biomarkers_4 = pd.read_csv("biomarkers_3.csv")
biomarkers = pd.concat([biomarkers_1,biomarkers_2, biomarkers_3, biomarkers_4])
targets = pd.read_csv("targets.csv")
with open('patient_profiles.json') as f:
   patient_profiles = json.load(f)
patient_profiles = pd.json_normalize(patient_profiles, record_path =['patient_profiles'])

Making the assumption that we need both patient_id and biomarker_id to be able to provide the patient the needed information.

In [3]:
all_data = pd.merge(targets, patient_profiles,on='patient_id')
all_data = pd.merge(all_data, biomarkers,on='biomarker_id')
all_data

,patient_id,biomarker_id,target_label,demographics.gender,demographics.age,status.disease_sub_type,status.comorbidity_index,status.cohort_qualifier,status.smoking_status,status.months_since_diagnosis,...,BM15147,BM15148,BM15149,BM15150,BM15151,BM15152,BM15153,BM15154,BM15155,BM15156
0,1010441f,3ae31327,0,MALE,52.0,A,NaN,True,NEVER,0.0,...,1,0,0,0,1,0,1,1.0,1,1
1,1010441f,3ae31327,0,MALE,52.0,A,NaN,True,NEVER,0.0,...,1,0,0,0,1,0,1,1.0,1,1
2,101eb6af1,1d8f7bab9,0,Female,55.0,B,NaN,True,never,0.0,...,1,0,1,0,0,0,1,NaN,1,1
3,10204394e,129ee8c6c,0,Male,69.0,A,NaN,True,never,9.0,...,0,0,1,0,0,1,1,1.0,1,1
4,1021d329b,1b44145bd,0,Male,52.0,A,NaN,True,former,10.0,...,1,0,0,0,0,1,1,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,f935591e,5f456337,1,Male,60.0,A,NaN,True,never,9.0,...,1,0,0,1,0,1,1,1.0,1,1
1772,fa4be3df,1b4e0b08f,1,Female,56.0,A,NaN,True,never,31.0,...,1,0,1,0,1,0,1,NaN,1,1
1773,fdaeb5f1,222e21be2,1,Female,66.0,A,NaN,True,unknown,10.0,...,0,0,1,1,0,0,1,NaN,1,1
1774,fdaeb5f1,222e21be2,1,Female,66.0,A,NaN,True,unknown,10.0,...,0,0,1,1,0,0,1,NaN,1,1


Review the biomarkers and patient profiles

In [4]:
all_data  = clean_columns(all_data)
all_data

,patient_id,biomarker_id,target_label,demographics_gender,demographics_age,status_disease_sub_type,status_comorbidity_index,status_cohort_qualifier,status_smoking_status,status_months_since_diagnosis,...,bm_15147,bm_15148,bm_15149,bm_15150,bm_15151,bm_15152,bm_15153,bm_15154,bm_15155,bm_15156
0,1010441f,3ae31327,0,male,52.0,A,NaN,True,NEVER,0.0,...,1,0,0,0,1,0,1,1.0,1,1
1,1010441f,3ae31327,0,male,52.0,A,NaN,True,NEVER,0.0,...,1,0,0,0,1,0,1,1.0,1,1
2,101eb6af1,1d8f7bab9,0,female,55.0,B,NaN,True,never,0.0,...,1,0,1,0,0,0,1,NaN,1,1
3,10204394e,129ee8c6c,0,male,69.0,A,NaN,True,never,9.0,...,0,0,1,0,0,1,1,1.0,1,1
4,1021d329b,1b44145bd,0,male,52.0,A,NaN,True,former,10.0,...,1,0,0,0,0,1,1,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,f935591e,5f456337,1,male,60.0,A,NaN,True,never,9.0,...,1,0,0,1,0,1,1,1.0,1,1
1772,fa4be3df,1b4e0b08f,1,female,56.0,A,NaN,True,never,31.0,...,1,0,1,0,1,0,1,NaN,1,1
1773,fdaeb5f1,222e21be2,1,female,66.0,A,NaN,True,unknown,10.0,...,0,0,1,1,0,0,1,NaN,1,1
1774,fdaeb5f1,222e21be2,1,female,66.0,A,NaN,True,unknown,10.0,...,0,0,1,1,0,0,1,NaN,1,1


Check descriptives for each variable.  Need to make sure variables are categorized the same.

In [21]:
#all_data.select_dtypes(include=['object']).drop(columns=['patient_id', 'biomarker_id']).value_counts()
for c in all_data.select_dtypes(include=['object']).drop(columns=['patient_id', 'biomarker_id']):
  print(all_data[c].value_counts())
  


male      983
female    793
Name: demographics_gender, dtype: int64
A    1191
B     261
C     167
F      53
D      52
E      52
Name: status_disease_sub_type, dtype: int64
never      624
current    352
former     275
NEVER      152
CURRENT    103
unknown     69
FORMER      56
UNKNOWN     16
Name: status_smoking_status, dtype: int64
White                                        581
Black or African American                    285
WHITE                                        177
Asian                                        142
BLACK OR AFRICAN AMERICAN                     52
ASIAN                                         30
Native Hawaiian or Other Pacific Islander     16
American Indian or Alaska Native              11
NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER      6
AMERICAN INDIAN OR ALASKA NATIVE               2
Name: demographics_race, dtype: int64
MODERATE    141
LOW         115
NONE         51
HIGH         20
Name: status_alcohol_usage, dtype: int64
normal         131
overweight    

In [11]:
y = all_data.target_label


def dummies_fun(x):
  x_dummies = pd.get_dummies(x.demographics_gender, prefix = "demographics_gender")
  x = pd.concat([x, x_dummies], axis = 1)
  return x

def filter_fun(x): 
  x = x.drop(columns = ['demographics_gender']) 
  return x

x_clean = (all_data.pipe(dummies_fun)
                  .pipe(filter_fun))
x_clean

,patient_id,biomarker_id,target_label,demographics_age,status_disease_sub_type,status_comorbidity_index,status_cohort_qualifier,status_smoking_status,status_months_since_diagnosis,demographics_race,...,bm_15149,bm_15150,bm_15151,bm_15152,bm_15153,bm_15154,bm_15155,bm_15156,demographics_gender_female,demographics_gender_male
0,1010441f,3ae31327,0,52.0,A,NaN,True,NEVER,0.0,BLACK OR AFRICAN AMERICAN,...,0,0,1,0,1,1.0,1,1,0,1
1,1010441f,3ae31327,0,52.0,A,NaN,True,NEVER,0.0,BLACK OR AFRICAN AMERICAN,...,0,0,1,0,1,1.0,1,1,0,1
2,101eb6af1,1d8f7bab9,0,55.0,B,NaN,True,never,0.0,NaN,...,1,0,0,0,1,NaN,1,1,1,0
3,10204394e,129ee8c6c,0,69.0,A,NaN,True,never,9.0,White,...,1,0,0,1,1,1.0,1,1,0,1
4,1021d329b,1b44145bd,0,52.0,A,NaN,True,former,10.0,Asian,...,0,0,0,1,1,1.0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,f935591e,5f456337,1,60.0,A,NaN,True,never,9.0,White,...,0,1,0,1,1,1.0,1,1,0,1
1772,fa4be3df,1b4e0b08f,1,56.0,A,NaN,True,never,31.0,NaN,...,1,0,1,0,1,NaN,1,1,1,0
1773,fdaeb5f1,222e21be2,1,66.0,A,NaN,True,unknown,10.0,White,...,1,1,0,0,1,NaN,1,1,1,0
1774,fdaeb5f1,222e21be2,1,66.0,A,NaN,True,unknown,10.0,White,...,1,1,0,0,1,NaN,1,1,1,0
